In [32]:
import pandas as pd

#file_path = "sim_squat_data.csv"
file_path = "../simulate_data/squat_data.csv"

df = pd.read_csv(file_path)

print(df.head())

   updown       UWB     vel_x     vel_y     vel_z     accel_x     accel_y  \
0       0  1.711766  0.000000  0.000000  0.000000   11.828938  -47.956334   
1       0  1.711567  0.032885 -0.103377  0.085973  161.323707  -58.555513   
2       0  1.707675  0.814416 -0.361158 -0.095207 -178.527859  131.141650   
3       0  1.706877 -0.014801  0.076857 -0.414512  167.629076 -131.972570   
4       0  1.702809  0.778336 -0.387729 -0.164145 -152.167755   50.021166   

      accel_z    gyro_x    gyro_y    gyro_z  
0   62.560536  0.000000  0.000000  0.000000  
1  -18.781445  1.277550  0.196830 -0.315507  
2 -121.957382  0.458584  2.095545 -2.194921  
3  115.180361 -2.522667  1.956186  0.759899  
4   -3.156194  1.050402  2.661206 -2.515833  


In [33]:
tmp = df['updown'].shift(1).fillna(0)
start_idxs = [0] + df[(tmp == 1) & (df['updown'] == 0)].index.tolist()

In [34]:
# 스쿼트 각 세트당 시작 행 삭제
for i in range(len(start_idxs)):
    start = start_idxs[i]
    
    df.drop(index=start, inplace=True)

In [35]:
# madgwick filter 사용해서 world frame으로 변환

from ahrs.filters import Madgwick
import numpy as np
from scipy.spatial.transform import Rotation as R

tmp = df['updown'].shift(1).fillna(0)
start_idxs = [0] + df[(tmp == 1) & (df['updown'] == 0)].index.tolist()

squat_sets = []

madgwick = Madgwick()

for i in range(len(start_idxs)):
    start = start_idxs[i]
    end = start_idxs[i + 1] if i + 1 < len(start_idxs) else len(df)

    squat = df.iloc[start:end].copy()
    quat = np.array([1.0, 0.0, 0.0, 0.0])

    for j in range(len(squat)):
        acc = squat.iloc[j][['accel_x', 'accel_y', 'accel_z']].values
        gyro = squat.iloc[j][['gyro_x', 'gyro_y', 'gyro_z']].values
        vel = squat.iloc[j][['vel_x', 'vel_y', 'vel_z']].values

        quat = madgwick.updateIMU(q=quat, gyr=gyro, acc=acc)
        rot_mat = R.from_quat([quat[1], quat[2], quat[3], quat[0]]).as_matrix()

        vel_world = rot_mat @ vel
        acc_world = rot_mat @ acc
        gyro_world = rot_mat @ gyro

        squat.iloc[j, 2:5] = vel_world
        squat.iloc[j, 5:8] = acc_world
        squat.iloc[j, 8:11] = gyro_world

    squat_sets.append(squat)

In [36]:
df = pd.concat(squat_sets, ignore_index=True)

In [37]:
print(df.head())

   updown       UWB     vel_x     vel_y     vel_z     accel_x     accel_y  \
0       0  1.711567  0.032672 -0.104551  0.084624  161.112580  -58.822470   
1       0  1.707675  0.802692 -0.379465 -0.120274 -177.896889  137.677035   
2       0  1.706877 -0.031034  0.074030 -0.414129  170.117828 -133.959633   
3       0  1.702809  0.747964 -0.419588 -0.218128 -149.761732   56.418053   
4       0  1.701773  0.015790 -0.146208 -0.195633  141.994315  -41.375078   

      accel_z    gyro_x    gyro_y    gyro_z  
0  -19.735317  1.277746  0.196754 -0.314760  
1 -115.506116  0.461164  2.122345 -2.168471  
2  109.075767 -2.455151  2.004541  0.849710  
3    7.451946  0.985855  2.624777 -2.579422  
4  -30.795562 -1.178501  0.701131  0.296573  


In [38]:
# UWB는 각 세트별로 정규화 (Min-Max Scaling) => 상대적인 거리가 중요
# IMU는 전체 데이터에서 정규화 (Standard Scaling) => 값들의 크기 자체가 의미가 있기 때문
# 얼마나 빨리, 세게 움직이는지 중요 따라서 크기 자체가 중요 (강도 차이 유지, 움직임 패턴이 왜곡 없이 유지)
from sklearn.preprocessing import StandardScaler, MinMaxScaler

imu_cols = ['vel_x', 'vel_y', 'vel_z', 
            'accel_x', 'accel_y', 'accel_z', 
            'gyro_x', 'gyro_y', 'gyro_z']
UWB_cols = ['UWB']
tmp = df['updown'].shift(1).fillna(0)
start_idxs = [0] + df[(tmp == 1) & (df['updown'] == 0)].index.tolist()

squat_sets = []

for i in range(len(start_idxs)):
    start = start_idxs[i]
    end = start_idxs[i + 1] if i + 1 < len(start_idxs) else len(df)

    squat = df.iloc[start:end].copy()
    if squat.empty:
        print(f"end == start: {end} == {start}")
        continue
    imu_scaler = StandardScaler()
    uwb_scaler = MinMaxScaler()

    squat[imu_cols] = imu_scaler.fit_transform(squat[imu_cols])
    squat[UWB_cols] = uwb_scaler.fit_transform(squat[UWB_cols])

    squat_sets.append(squat)

In [39]:
# smoothing imu data

for i in range(len(squat_sets)):
    squat = squat_sets[i]
    squat[imu_cols] = squat[imu_cols].rolling(window=15, min_periods=15).mean()
    squat_sets[i] = squat

In [40]:
df_normalized = pd.concat(squat_sets, ignore_index=True)

In [41]:
df_normalized = df_normalized.dropna()

In [42]:
df_normalized.head()

,updown,UWB,vel_x,vel_y,vel_z,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
14,0,0.591120,0.444669,-0.467373,-0.315034,0.023474,-0.032762,-0.094302,0.133845,0.804320,-0.486455
15,0,0.573637,0.480935,-0.528851,-0.349580,-0.163544,0.073112,-0.082049,0.149239,0.865470,-0.586657
16,0,0.567564,0.402824,-0.462519,-0.375447,0.023747,-0.066839,-0.033649,0.073584,0.806644,-0.486584
17,0,0.549405,0.436299,-0.535916,-0.401144,-0.157985,0.071472,-0.074418,0.248877,0.810755,-0.600475
18,0,0.542702,0.360260,-0.461036,-0.426632,-0.003370,0.012905,-0.142077,0.133730,0.725031,-0.480376


In [43]:
df_normalized.to_csv("sim_squat_data.csv", index=False)